In [33]:
from random import randint, randrange


class Node:
    def __init__(self, leaf=False):
        self.leaf = leaf
        self.keys = []
        self.children = []


class BTree:
    def __init__(self, t):
        """
        Initializing the B-Tree
        :param t: Order.
        """
        self.root = Node(True)
        self.t = t

    def printTree(self, x, lists, lvl=0):
        """
        Prints the complete B-Tree
        :param x: Root node.
        :param lvl: Current level.
        """
        #print(lvl,end=" ")
        lvl += 1
        if len(x.children) > 0:
            for i in x.children:
                self.printTree(i, lists, lvl)
        lists.append(lvl)

    def search(self, k, x=None):
        """
        Search for key 'k' at position 'x'
        :param k: The key to search for.
        :param x: The position to search from. If not specified, then search occurs from the root.
        :return: 'None' if 'k' is not found. Otherwise returns a tuple of (node, index) at which the key was found.
        """
        if x is not None:
            i = 0
            while i < len(x.keys) and k > x.keys[i][0]:
                i += 1
            if i < len(x.keys) and k == x.keys[i][0]:
                return x, i
            elif x.leaf:
                return None
            else:
                # Search its children
                return self.search(k, x.children[i])
        else:
            # Search the entire tree
            return self.search(k, self.root)

    def insert(self, k):
        """
        Calls the respective helper functions for insertion into B-Tree
        :param k: The key to be inserted.
        """
        root = self.root
        # If a node is full, split the child
        if len(root.keys) == (2 * self.t) - 1:
            temp = Node()
            self.root = temp
            # Former root becomes 0'th child of new root 'temp'
            temp.children.insert(0, root)
            self._splitChild(temp, 0)
            self._insertNonFull(temp, k)
        else:
            self._insertNonFull(root, k)

    def _insertNonFull(self, x, k):
        """
        Inserts a key in a non-full node
        :param x: The key to be inserted.
        :param k: The position of node.
        """
        i = len(x.keys) - 1
        if x.leaf:
            x.keys.append((None, None))
            while i >= 0 and k[0] < x.keys[i][0]:
                x.keys[i + 1] = x.keys[i]
                i -= 1
            x.keys[i + 1] = k
        else:
            while i >= 0 and k[0] < x.keys[i][0]:
                i -= 1
            i += 1
            if len(x.children[i].keys) == (2 * self.t) - 1:
                self._splitChild(x, i)
                if k[0] > x.keys[i][0]:
                    i += 1
            self._insertNonFull(x.children[i], k)

    def _splitChild(self, x, i):
        """
        Splits the child of node at 'x' from index 'i'
        :param x: Parent node of the node to be split.
        :param i: Index value of the child.
        """
        t = self.t
        y = x.children[i]
        z = Node(y.leaf)
        x.children.insert(i + 1, z)
        x.keys.insert(i, y.keys[t - 1])
        z.keys = y.keys[t: (2 * t) - 1]
        y.keys = y.keys[0: t - 1]
        if not y.leaf:
            z.children = y.children[t: 2 * t]
            y.children = y.children[0: t - 1]

    def delete(self, x, k):
        """
        Calls the respective helper functions for deletion from B-Tree
        :param x: The node, according to whose relative position, helper functions are called.
        :param k: The key to be deleted.
        """
        t = self.t
        i = 0
        while i < len(x.keys) and k[0] > x.keys[i][0]:
            i += 1
        # Deleting the key if the node is a leaf
        if x.leaf:
            if i < len(x.keys) and x.keys[i][0] == k[0]:
                x.keys.pop(i)
                return
            return

        # Calling '_deleteInternalNode' when x is an internal node and contains the key 'k'
        if i < len(x.keys) and x.keys[i][0] == k[0]:
            return self._deleteInternalNode(x, k, i)
        # Recursively calling 'delete' on x's children
        elif len(x.children[i].keys) >= t:
            self.delete(x.children[i], k)
        # Ensuring that a child always has atleast 't' keys
        else:
            if i != 0 and i + 2 < len(x.children):
                if len(x.children[i - 1].keys) >= t:
                    self._deleteSibling(x, i, i - 1)
                elif len(x.children[i + 1].keys) >= t:
                    self._deleteSibling(x, i, i + 1)
                else:
                    self._deleteMerge(x, i, i + 1)
            elif i == 0:
                if len(x.children[i + 1].keys) >= t:
                    self._deleteSibling(x, i, i + 1)
                else:
                    self._deleteMerge(x, i, i + 1)
            elif i + 1 == len(x.children):
                if len(x.children[i - 1].keys) >= t:
                    self._deleteSibling(x, i, i - 1)
                else:
                    self._deleteMerge(x, i, i - 1)
            self.delete(x.children[i], k)

    def _deleteInternalNode(self, x, k, i):
        """
        Deletes internal node
        :param x: The internal node in which key 'k' is present.
        :param k: The key to be deleted.
        :param i: The index position of key in the list
        """
        t = self.t
        # Deleting the key if the node is a leaf
        if x.leaf:
            if x.keys[i][0] == k[0]:
                x.keys.pop(i)
                return
            return

        # Replacing the key with its predecessor and deleting predecessor
        if len(x.children[i].keys) >= t:
            x.keys[i] = self._deletePredecessor(x.children[i])
            return
        # Replacing the key with its successor and deleting successor
        elif len(x.children[i + 1].keys) >= t:
            x.keys[i] = self._deleteSuccessor(x.children[i + 1])
            return
        # Merging the child, its left sibling and the key 'k'
        else:
            self._deleteMerge(x, i, i + 1)
            self._deleteInternalNode(x.children[i], k, self.t - 1)

    def _deletePredecessor(self, x):
        """
        Deletes predecessor of key 'k' which is to be deleted
        :param x: Node
        :return: Predecessor of key 'k' which is to be deleted
        """
        if x.leaf:
            return x.keys.pop()
        n = len(x.keys) - 1
        if len(x.children[n].keys) >= self.t:
            self._deleteSibling(x, n + 1, n)
        else:
            self._deleteMerge(x, n, n + 1)
        self._deletePredecessor(x.children[n])

    def _deleteSuccessor(self, x):
        """
        Deletes successor of key 'k' which is to be deleted
        :param x: Node
        :return: Successor of key 'k' which is to be deleted
        """
        if x.leaf:
            return x.keys.pop(0)
        if len(x.children[1].keys) >= self.t:
            self._deleteSibling(x, 0, 1)
        else:
            self._deleteMerge(x, 0, 1)
        self._deleteSuccessor(x.children[0])

    def _deleteMerge(self, x, i, j):
        """
        Merges the children of x and one of its own keys
        :param x: Parent node
        :param i: The index of one of the children
        :param j: The index of one of the children
        """
        cNode = x.children[i]

        # Merging the x.children[i], x.children[j] and x.keys[i]
        if j > i:
            rsNode = x.children[j]
            cNode.keys.append(x.keys[i])
            # Assigning keys of right sibling node to child node
            for k in range(len(rsNode.keys)):
                cNode.keys.append(rsNode.keys[k])
                if len(rsNode.children) > 0:
                    cNode.children.append(rsNode.children[k])
            if len(rsNode.children) > 0:
                cNode.children.append(rsNode.children.pop())
            new = cNode
            x.keys.pop(i)
            x.children.pop(j)
        # Merging the x.children[i], x.children[j] and x.keys[i]
        else:
            lsNode = x.children[j]
            lsNode.keys.append(x.keys[j])
            # Assigning keys of left sibling node to child node
            for i in range(len(cNode.keys)):
                lsNode.keys.append(cNode.keys[i])
                if len(lsNode.children) > 0:
                    lsNode.children.append(cNode.children[i])
            if len(lsNode.children) > 0:
                lsNode.children.append(cNode.children.pop())
            new = lsNode
            x.keys.pop(j)
            x.children.pop(i)

        # If x is root and is empty, then re-assign root
        if x == self.root and len(x.keys) == 0:
            self.root = new

    @staticmethod
    def _deleteSibling(x, i, j):
        """
        Borrows a key from j'th child of x and appends it to i'th child of x
        :param x: Parent node
        :param i: The index of one of the children
        :param j: The index of one of the children
        """
        cNode = x.children[i]
        if i < j:
            # Borrowing key from right sibling of the child
            rsNode = x.children[j]
            cNode.keys.append(x.keys[i])
            x.keys[i] = rsNode.keys[0]
            if len(rsNode.children) > 0:
                cNode.children.append(rsNode.children[0])
                rsNode.children.pop(0)
            rsNode.keys.pop(0)
        else:
            # Borrowing key from left sibling of the child
            lsNode = x.children[j]
            cNode.keys.insert(0, x.keys[i - 1])
            x.keys[i - 1] = lsNode.keys.pop()
            if len(lsNode.children) > 0:
                cNode.children.insert(0, lsNode.children.pop())



In [56]:
import time
import numpy as np

NUM = 1000
timeSum = []
heightSum = []
for i in range(10):

    t = BTree(5)
    lists = []
    f = open(str(NUM)+"_"+str(i)+".txt",'r')
    nums = []
    lines = f.readlines()
    for line in lines:
        nums.append(int(line))

    start = time.time()
    
    for num in range(NUM):
        t.insert((num, nums[num]))
    timeSum.append(time.time() - start)
    print(i+1,"번째 트리 생성 시간: ",time.time() - start)
    t.printTree(t.root,lists)
    print(i+1,"번째 트리 높이: ", max(lists))

    heightSum.append(max(lists))
    print()
    for j in range(5):
        start = time.time()
        randNum = nums[np.random.randint(NUM)]
        t.delete(t.root,(randNum,))
        print(i+1,"-",j+1,"번째 탐색, 찾는 숫자",randNum,"탐색+삭제 시간:",time.time() - start)
    
    print()
    
    f.close()

print(np.mean(timeSum))
print(np.mean(heightSum))

1 번째 트리 생성 시간:  0.003033876419067383
1 번째 트리 높이:  4

1 - 1 번째 탐색, 찾는 숫자 1491805 탐색+삭제 시간: 0.0
1 - 2 번째 탐색, 찾는 숫자 5460555 탐색+삭제 시간: 0.0
1 - 3 번째 탐색, 찾는 숫자 7898021 탐색+삭제 시간: 0.0009548664093017578
1 - 4 번째 탐색, 찾는 숫자 9517524 탐색+삭제 시간: 0.0
1 - 5 번째 탐색, 찾는 숫자 3963995 탐색+삭제 시간: 0.0

2 번째 트리 생성 시간:  0.0019936561584472656
2 번째 트리 높이:  4

2 - 1 번째 탐색, 찾는 숫자 8889600 탐색+삭제 시간: 0.0
2 - 2 번째 탐색, 찾는 숫자 8466818 탐색+삭제 시간: 0.0
2 - 3 번째 탐색, 찾는 숫자 8573974 탐색+삭제 시간: 0.0
2 - 4 번째 탐색, 찾는 숫자 3757732 탐색+삭제 시간: 0.0
2 - 5 번째 탐색, 찾는 숫자 8512945 탐색+삭제 시간: 0.000997304916381836

3 번째 트리 생성 시간:  0.002995014190673828
3 번째 트리 높이:  4

3 - 1 번째 탐색, 찾는 숫자 9162204 탐색+삭제 시간: 0.0
3 - 2 번째 탐색, 찾는 숫자 8638016 탐색+삭제 시간: 0.0
3 - 3 번째 탐색, 찾는 숫자 5236415 탐색+삭제 시간: 0.0
3 - 4 번째 탐색, 찾는 숫자 6121793 탐색+삭제 시간: 0.0
3 - 5 번째 탐색, 찾는 숫자 4324111 탐색+삭제 시간: 0.0

4 번째 트리 생성 시간:  0.002037525177001953
4 번째 트리 높이:  4

4 - 1 번째 탐색, 찾는 숫자 4474025 탐색+삭제 시간: 0.0
4 - 2 번째 탐색, 찾는 숫자 9255278 탐색+삭제 시간: 0.0
4 - 3 번째 탐색, 찾는 숫자 7777796 탐색+삭제 시간: 0.0
4 - 4 번째 탐색

In [57]:
import time
import numpy as np

NUM = 10000

for i in range(10):

    t = BTree(5)
    lists = []
    f = open(str(NUM)+"_"+str(i)+".txt",'r')
    nums = []
    lines = f.readlines()
    for line in lines:
        nums.append(int(line))

    start = time.time()
    
    for num in range(NUM):
        t.insert((num, nums[num]))
    
    timeSum.append(time.time() - start)
    print(i+1,"번째 트리 생성 시간: ",time.time() - start)
    t.printTree(t.root,lists)
    print(i+1,"번째 트리 높이: ", max(lists))
    heightSum.append(max(lists))
    print()
    for j in range(5):
        start = time.time()
        randNum = nums[np.random.randint(NUM)]
        
        try:
            t.delete(t.root,(1*j,))
            print(i+1,"-",j+1,"번째 탐색, 찾는 숫자",randNum,"탐색+삭제 시간:",time.time() - start)

        except IndexError:
            print("")
    
    print()
    
    f.close()

print(np.mean(timeSum))
print(np.mean(heightSum))

1 번째 트리 생성 시간:  0.03889322280883789
1 번째 트리 높이:  6

1 - 1 번째 탐색, 찾는 숫자 3564143 탐색+삭제 시간: 0.0
1 - 2 번째 탐색, 찾는 숫자 2680778 탐색+삭제 시간: 0.0
1 - 3 번째 탐색, 찾는 숫자 959516 탐색+삭제 시간: 0.0
1 - 4 번째 탐색, 찾는 숫자 1605218 탐색+삭제 시간: 0.0
1 - 5 번째 탐색, 찾는 숫자 3078124 탐색+삭제 시간: 0.0

2 번째 트리 생성 시간:  0.03657793998718262
2 번째 트리 높이:  6

2 - 1 번째 탐색, 찾는 숫자 2927502 탐색+삭제 시간: 0.0
2 - 2 번째 탐색, 찾는 숫자 7058612 탐색+삭제 시간: 0.0
2 - 3 번째 탐색, 찾는 숫자 806808 탐색+삭제 시간: 0.0
2 - 4 번째 탐색, 찾는 숫자 4328186 탐색+삭제 시간: 0.0
2 - 5 번째 탐색, 찾는 숫자 4449545 탐색+삭제 시간: 0.0

3 번째 트리 생성 시간:  0.03126931190490723
3 번째 트리 높이:  6

3 - 1 번째 탐색, 찾는 숫자 2526467 탐색+삭제 시간: 0.0
3 - 2 번째 탐색, 찾는 숫자 7029256 탐색+삭제 시간: 0.0
3 - 3 번째 탐색, 찾는 숫자 4389277 탐색+삭제 시간: 0.0
3 - 4 번째 탐색, 찾는 숫자 2651028 탐색+삭제 시간: 0.0
3 - 5 번째 탐색, 찾는 숫자 8142720 탐색+삭제 시간: 0.0

4 번째 트리 생성 시간:  0.09372830390930176
4 번째 트리 높이:  6

4 - 1 번째 탐색, 찾는 숫자 1386273 탐색+삭제 시간: 0.0
4 - 2 번째 탐색, 찾는 숫자 7762123 탐색+삭제 시간: 0.0
4 - 3 번째 탐색, 찾는 숫자 7151407 탐색+삭제 시간: 0.0
4 - 4 번째 탐색, 찾는 숫자 4754599 탐색+삭제 시간: 0.0
4 - 5 번째 탐색,

In [58]:
import time
import numpy as np

NUM = 100000
timeSum = []
heightSum = []
for i in range(10):

    t = BTree(5)
    lists = []
    f = open(str(NUM)+"_"+str(i)+".txt",'r')
    nums = []
    lines = f.readlines()
    for line in lines:
        nums.append(int(line))

    start = time.time()
    
    for num in range(NUM):
        t.insert((num, nums[num]))
    timeSum.append(time.time() - start)
    print(i+1,"번째 트리 생성 시간: ",time.time() - start)
    t.printTree(t.root,lists)
    print(i+1,"번째 트리 높이: ", max(lists))

    heightSum.append(max(lists))
    print()
    for j in range(5):
        start = time.time()
        randNum = nums[np.random.randint(NUM)]
        t.delete(t.root,(randNum,))
        print(i+1,"-",j+1,"번째 탐색, 찾는 숫자",randNum,"탐색+삭제 시간:",time.time() - start)
    
    print()
    
    f.close()

print(np.mean(timeSum))
print(np.mean(heightSum))

1 번째 트리 생성 시간:  0.4370887279510498
1 번째 트리 높이:  7

1 - 1 번째 탐색, 찾는 숫자 5675115 탐색+삭제 시간: 0.0
1 - 2 번째 탐색, 찾는 숫자 7364462 탐색+삭제 시간: 0.0
1 - 3 번째 탐색, 찾는 숫자 3267925 탐색+삭제 시간: 0.0
1 - 4 번째 탐색, 찾는 숫자 419345 탐색+삭제 시간: 0.0
1 - 5 번째 탐색, 찾는 숫자 3101112 탐색+삭제 시간: 0.0

2 번째 트리 생성 시간:  0.4618649482727051
2 번째 트리 높이:  7

2 - 1 번째 탐색, 찾는 숫자 5169110 탐색+삭제 시간: 0.0
2 - 2 번째 탐색, 찾는 숫자 2542695 탐색+삭제 시간: 0.0
2 - 3 번째 탐색, 찾는 숫자 4013536 탐색+삭제 시간: 0.0
2 - 4 번째 탐색, 찾는 숫자 7967028 탐색+삭제 시간: 0.0
2 - 5 번째 탐색, 찾는 숫자 7588188 탐색+삭제 시간: 0.0

3 번째 트리 생성 시간:  0.43784499168395996
3 번째 트리 높이:  7

3 - 1 번째 탐색, 찾는 숫자 1822991 탐색+삭제 시간: 0.0
3 - 2 번째 탐색, 찾는 숫자 4383861 탐색+삭제 시간: 0.0
3 - 3 번째 탐색, 찾는 숫자 435523 탐색+삭제 시간: 0.0
3 - 4 번째 탐색, 찾는 숫자 927145 탐색+삭제 시간: 0.0
3 - 5 번째 탐색, 찾는 숫자 2990303 탐색+삭제 시간: 0.0

4 번째 트리 생성 시간:  0.5018162727355957
4 번째 트리 높이:  7

4 - 1 번째 탐색, 찾는 숫자 2764429 탐색+삭제 시간: 0.0
4 - 2 번째 탐색, 찾는 숫자 8916102 탐색+삭제 시간: 0.0
4 - 3 번째 탐색, 찾는 숫자 3274835 탐색+삭제 시간: 0.0
4 - 4 번째 탐색, 찾는 숫자 3575313 탐색+삭제 시간: 0.0
4 - 5 번째 탐색, 찾는 

In [65]:
import time
import numpy as np

NUM = 1000000
timeSum = []
heightSum = []
for i in range(10):

    t = BTree(5)
    lists = []
    f = open(str(NUM)+"_"+str(i)+".txt",'r')
    nums = []
    lines = f.readlines()
    for line in lines:
        nums.append(int(line))

    start = time.time()
    
    for num in range(NUM):
        t.insert((num, nums[num]))
    timeSum.append(time.time() - start)
    print(i+1,"번째 트리 생성 시간: ",time.time() - start)
    t.printTree(t.root,lists)
    print(i+1,"번째 트리 높이: ", max(lists))
    heightSum.append(max(lists))
    print()
    for j in range(5):
        start = time.time()
        randNum = nums[np.random.randint(0,10)]
        t.delete(t.root,(randNum,))
        print(i+1,"-",j+1,"번째 탐색, 찾는 숫자",randNum,"탐색+삭제 시간:",time.time() - start)
    
    print()
    
    f.close()

print(np.mean(timeSum))
print(np.mean(heightSum))

1 번째 트리 생성 시간:  5.552413702011108
1 번째 트리 높이:  9

1 - 1 번째 탐색, 찾는 숫자 5948625 탐색+삭제 시간: 0.0
1 - 2 번째 탐색, 찾는 숫자 5948625 탐색+삭제 시간: 0.0
1 - 3 번째 탐색, 찾는 숫자 4996931 탐색+삭제 시간: 0.0
1 - 4 번째 탐색, 찾는 숫자 6767725 탐색+삭제 시간: 0.0
1 - 5 번째 탐색, 찾는 숫자 4842280 탐색+삭제 시간: 0.0

2 번째 트리 생성 시간:  5.591880559921265
2 번째 트리 높이:  9

2 - 1 번째 탐색, 찾는 숫자 8360281 탐색+삭제 시간: 0.0
2 - 2 번째 탐색, 찾는 숫자 2226784 탐색+삭제 시간: 0.0
2 - 3 번째 탐색, 찾는 숫자 9708415 탐색+삭제 시간: 0.0
2 - 4 번째 탐색, 찾는 숫자 2226784 탐색+삭제 시간: 0.0
2 - 5 번째 탐색, 찾는 숫자 2124963 탐색+삭제 시간: 0.0

3 번째 트리 생성 시간:  5.223110198974609
3 번째 트리 높이:  9

3 - 1 번째 탐색, 찾는 숫자 9599093 탐색+삭제 시간: 0.0
3 - 2 번째 탐색, 찾는 숫자 8857142 탐색+삭제 시간: 0.0
3 - 3 번째 탐색, 찾는 숫자 1137233 탐색+삭제 시간: 0.0
3 - 4 번째 탐색, 찾는 숫자 3311720 탐색+삭제 시간: 0.0
3 - 5 번째 탐색, 찾는 숫자 5724768 탐색+삭제 시간: 0.0

4 번째 트리 생성 시간:  5.367174863815308
4 번째 트리 높이:  9

4 - 1 번째 탐색, 찾는 숫자 5778654 탐색+삭제 시간: 0.0
4 - 2 번째 탐색, 찾는 숫자 2101946 탐색+삭제 시간: 0.0
4 - 3 번째 탐색, 찾는 숫자 8397238 탐색+삭제 시간: 0.0
4 - 4 번째 탐색, 찾는 숫자 4754851 탐색+삭제 시간: 0.0
4 - 5 번째 탐색, 찾는 숫자